In [10]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import constants
import IPython.display as ipd
import os
from os import listdir
from os.path import isfile, join
import shutil
import IPython
import matplotlib
import seaborn as sns
from scipy.io import wavfile
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, Embedding,Conv3D,Bidirectional
from keras.layers import (Convolution1D, Convolution2D,Convolution3D,GlobalAveragePooling1D,GlobalAveragePooling2D, 
                          BatchNormalization, Flatten,
                          GlobalMaxPool1D,LeakyReLU,GlobalMaxPool3D,
                          MaxPool1D,MaxPool2D,MaxPool3D, Flatten, concatenate, Activation,ELU)
from keras.utils import Sequence, to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import optimizers
from sklearn.model_selection import train_test_split

In [11]:
np.random.seed(1001)

In [12]:
train_wav_loc = "../audio_train/"
test_wav_loc = "../audio_test/"

In [13]:
dataset = pd.read_csv("train.csv",header=0,names=['name','label','verified'])
verifiedData = dataset.loc[dataset.verified == 1,["name","label"]]

In [6]:
len(set(verifiedData['label']))

41

In [14]:
sample_rate=44100
# Based on https://www.kaggle.com/codename007/a-very-extensive-freesound-exploratory-analysis?scriptVersionId=3019309
def load_wav_file(name):
    _, b = wavfile.read(train_wav_loc + name)
    assert _ == sample_rate
    return b

verifiedData['time_series'] = verifiedData['name'].apply(load_wav_file)
verifiedData['nframes'] = verifiedData['time_series'].apply(len)

In [13]:
def tanh_estimator(df):
    temp_df = []
    for i in df:
        temp_df.append(0.5*(np.tanh(0.01*(i-np.mean(i))/np.std(i))+ 1)) 
    return temp_df
    #return (df - df.mean(axis=0)) / (df.std(axis=0))

    
def getData(df, wav_path,train=True):
    dset=[]
    audiolen=2*sample_rate
    
    if train==False:
        ddir=data_loc+"audio_test/"
        files=[f for f in listdir(ddir) if isfile(join(testpath, f))]
    else:
        ddir=wav_path
        files=df.name.values
    
    for file in files:
        wav,_= librosa.core.load(ddir+file, sr=sample_rate)
        
        # Pad/trim WAV data
        if len(wav) > audiolen:
            wav = wav[:audiolen]
        elif len(wav) < audiolen:
            off=audiolen - len(wav) 
            wav=np.pad(wav, (off, audiolen - len(wav) - off), 'constant')
            
        mfcc = librosa.feature.mfcc(wav, sr = sample_rate, n_mfcc=40)
        dset.append(mfcc)
    
    return (tanh_estimator(np.array(dset))) # Return normalized dataset

In [15]:
def tanh_estimator(df):
    return (df - df.mean(axis=0)) / (df.std(axis=0))

    
def getData2(df, wav_path,train=True):
    dset=[]
    audiolen=2*sample_rate
    
    if train==False:
        ddir=data_loc+"audio_test/"
        files=[f for f in listdir(ddir) if isfile(join(testpath, f))]
    else:
        ddir=wav_path
        files=df.name.values
    
    for file in files:
        wav,_= librosa.core.load(ddir+file, sr=sample_rate)
        
        # Pad/trim WAV data
        if len(wav) > audiolen:
            wav = wav[:audiolen]
        elif len(wav) < audiolen:
            off=audiolen - len(wav) 
            wav=np.pad(wav, (off, audiolen - len(wav) - off), 'constant')
            
        mfcc = librosa.feature.melspectrogram(wav, sr = sample_rate)
        dset.append(mfcc)
    
    return (tanh_estimator(np.array(dset))) # Return normalized dataset

In [14]:
x_train1 = getData(verifiedData,wav_path=train_wav_loc)

[[[-5.51071725e+02 -5.48230601e+02 -5.50604943e+02 ... -3.13616739e+02
   -2.79579421e+02 -2.41716183e+02]
  [ 8.12112048e+01  8.35118348e+01  8.11529295e+01 ...  1.69817914e+02
    2.11521237e+02  2.28720407e+02]
  [ 7.49453576e+00  5.95448473e+00  3.43608728e+00 ... -6.87291967e+01
   -3.98055192e+01 -2.64086023e+01]
  ...
  [ 7.78239823e+00  6.46160817e+00  2.95491671e+00 ...  2.66865271e+01
    1.27676483e+01  3.97639425e+00]
  [ 2.75955950e+00  2.27997409e+00  1.55283725e+00 ... -1.69544508e+00
   -9.45686653e+00 -1.09585406e+01]
  [-2.87850891e-01 -1.16869237e+00 -7.46875017e-01 ... -1.77553427e+01
   -1.91719061e+01 -1.26074113e+01]]

 [[-2.66613791e+02 -3.14772225e+02 -4.56226166e+02 ... -7.67030993e+02
   -7.46648208e+02 -7.06407422e+02]
  [-1.30817772e+00  3.70002406e+00  8.90681810e+00 ...  2.85422603e+01
    5.06317229e+01  9.14606479e+01]
  [-1.43385265e+02 -1.33259042e+02 -9.78326385e+01 ... -5.03349569e-01
    5.98699270e+00  1.51138550e+01]
  ...
  [ 1.74359887e+00 -1.3

In [24]:
x_train2 = getData2(verifiedData,wav_path=train_wav_loc)

In [17]:
from sklearn import preprocessing

def encode(data, oneHot=False):
    # Encode labels into integers or onehot
    le = preprocessing.LabelEncoder()
    le.fit(data)
    trans = le.transform(data)
    
    if oneHot== True:
        ohlen = (len(set(trans)))
        eye=np.eye(ohlen)
        return to_categorical(trans)
        #return np.array([eye[i] for i in set(verifiedData['label'])])
    else:
        return trans

In [22]:
yt=encode(verifiedData.label.values, oneHot=True)
# Get validation set - keep 70% for train, 15% validation, 15% test
training_size = int(np.shape(x_train1)[0] * 0.7)
validate_size = int(np.shape(x_train1)[0]*.15)

x_train=x_train1[:training_size] # shape (2968, 40, 28)
x_val=x_train1[training_size:training_size+validate_size] # shape (742, 40, 28)
x_test=x_train1[training_size+validate_size:]

y_train=yt[:training_size] # shape 2968, 41
y_val=yt[training_size:training_size+validate_size] # shape 742, 41
y_test=yt[training_size+validate_size:]

print ('Train sizes:')
print (np.shape(x_train))
print (np.shape(y_train))

print ('\nValidation sizes:')
print (np.shape(x_val))
print (np.shape(y_val))

print ('\nTest sizes:')
print (np.shape(x_test))
print (np.shape(y_test))

Train sizes:
(2597, 40, 173)
(2597, 41)

Validation sizes:
(556, 40, 173)
(556, 41)

Test sizes:
(557, 40, 173)
(557, 41)


In [25]:
yt2=encode(verifiedData.label.values, oneHot=True)
# Get validation set - keep 70% for train, 15% validation, 15% test
training_size2 = int(np.shape(x_train2)[0] * 0.7)
validate_size2 = int(np.shape(x_train2)[0]*.15)

x_train3=x_train2[:training_size2] # shape (2968, 40, 28)
x_val2=x_train2[training_size2:training_size2+validate_size2] # shape (742, 40, 28)
x_test2=x_train2[training_size2+validate_size2:]

y_train3=yt2[:training_size2] # shape 2968, 41
y_val2=yt2[training_size2:training_size2+validate_size2] # shape 742, 41
y_test2=yt2[training_size2+validate_size2:]

print ('Train sizes:')
print (np.shape(x_train2))
print (np.shape(y_train2))

print ('\nValidation sizes:')
print (np.shape(x_val2))
print (np.shape(y_val2))

print ('\nTest sizes:')
print (np.shape(x_test2))
print (np.shape(y_test2))

Train sizes:
(3710, 128, 173)
(2597, 41)

Validation sizes:
(556, 128, 173)
(556, 41)

Test sizes:
(557, 128, 173)
(557, 41)


In [29]:
x_tr

(2597, 40, 173)

In [24]:
conv_model4 = Sequential([
    Convolution1D(256,20, padding="same", input_shape=(np.shape(x_train)[1], np.shape(x_train)[2]),\
                 activation='relu',strides=1),
    BatchNormalization(),
    MaxPool1D(3),
    
    Convolution1D(256,20, padding="same", input_shape=(np.shape(x_train)[1], np.shape(x_train)[2]),\
                 activation='relu',strides=3),
    BatchNormalization(),
    MaxPool1D(3),
    
    Bidirectional(LSTM(256, return_sequences=True)),
    Dense(173, activation='relu'),
    BatchNormalization(),
    
    Convolution1D(128,20, padding="same",strides=3),
    BatchNormalization(),
    GlobalAveragePooling1D(),
    Dropout(rate=0.3),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
    
])
conv_model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=10,\
                      verbose=0, mode='auto', baseline=None)
print (conv_model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 40, 256)           886016    
_________________________________________________________________
batch_normalization_1 (Batch (None, 40, 256)           1024      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 5, 256)            1310976   
_________________________________________________________________
batch_normalization_2 (Batch (None, 5, 256)            1024      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 256)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1, 512)            1050624   
__________

In [36]:
conv_model4.fit(np.array(x_train), np.array(y_train),
          batch_size=50, epochs=50,
          validation_data=(np.array(x_val), np.array(y_val)))

Train on 2597 samples, validate on 556 samples
Epoch 1/50
2597/2597 [==============================] - 15s 6ms/step - loss: 0.4781 - acc: 0.8568 - val_loss: 15.8862 - val_acc: 0.0144
Epoch 2/50
2597/2597 [==============================] - 13s 5ms/step - loss: 0.4598 - acc: 0.8664 - val_loss: 14.7556 - val_acc: 0.0845
Epoch 3/50
2597/2597 [==============================] - 13s 5ms/step - loss: 0.4221 - acc: 0.8818 - val_loss: 15.6543 - val_acc: 0.0288
Epoch 4/50
2597/2597 [==============================] - 14s 5ms/step - loss: 0.4205 - acc: 0.8787 - val_loss: 15.3848 - val_acc: 0.0378
Epoch 5/50
2597/2597 [==============================] - 13s 5ms/step - loss: 0.4661 - acc: 0.8637 - val_loss: 15.8572 - val_acc: 0.0162
Epoch 6/50
2597/2597 [==============================] - 14s 5ms/step - loss: 0.5144 - acc: 0.8363 - val_loss: 15.5383 - val_acc: 0.0360
Epoch 7/50
2597/2597 [==============================] - 13s 5ms/step - loss: 0.4624 - acc: 0.8633 - val_loss: 14.7556 - val_acc: 0.0845
E

In [37]:
conv_result4 = conv_model4.evaluate(np.array(x_test), np.array(y_test), batch_size=150)
print(conv_result4)

557/557 [==============================] - 1s 1ms/step
[15.770850791109314, 0.021543986209178315]


In [26]:
conv_model4 = Sequential([
    Convolution1D(256,20, padding="same", input_shape=(x_train2.shape[1], x_train2.shape[2]),\
                 activation='relu',strides=1),
    BatchNormalization(),
    MaxPool1D(3),
    
    Convolution1D(256,20, padding="same", input_shape=(x_train2.shape[1], x_train2.shape[2]),\
                 activation='relu',strides=3),
    BatchNormalization(),
    MaxPool1D(3),
    
    Bidirectional(LSTM(256, return_sequences=True)),
    Dense(173, activation='relu'),
    BatchNormalization(),
    
    Convolution1D(128,20, padding="same",strides=3),
    BatchNormalization(),
    GlobalAveragePooling1D(),
    Dropout(rate=0.3),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
    
])
conv_model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=10,\
                      verbose=0, mode='auto', baseline=None)
print (conv_model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 128, 256)          886016    
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 256)          1024      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 42, 256)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 14, 256)           1310976   
_________________________________________________________________
batch_normalization_6 (Batch (None, 14, 256)           1024      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 4, 256)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 4, 512)            1050624   
__________

In [59]:
conv_model4.fit(x_train3, y_train3,
          batch_size=75, epochs=40,
          validation_data=(x_val2, y_val2))
#87+10+10 0.456
#87+10+10+10+10 0.4057

Train on 2597 samples, validate on 556 samples
Epoch 1/40
2597/2597 [==============================] - 21s 8ms/step - loss: 0.1365 - acc: 0.9630 - val_loss: 6.3653 - val_acc: 0.3651
Epoch 2/40
2597/2597 [==============================] - 21s 8ms/step - loss: 0.1513 - acc: 0.9592 - val_loss: 4.5230 - val_acc: 0.3957
Epoch 3/40
2597/2597 [==============================] - 21s 8ms/step - loss: 0.1368 - acc: 0.9619 - val_loss: 4.7214 - val_acc: 0.3939
Epoch 4/40
2597/2597 [==============================] - 24s 9ms/step - loss: 0.1332 - acc: 0.9638 - val_loss: 5.4134 - val_acc: 0.3525
Epoch 5/40
2597/2597 [==============================] - 21s 8ms/step - loss: 0.1330 - acc: 0.9630 - val_loss: 4.9467 - val_acc: 0.3921
Epoch 6/40
2597/2597 [==============================] - 20s 8ms/step - loss: 0.1245 - acc: 0.9692 - val_loss: 5.3896 - val_acc: 0.3525
Epoch 7/40
2597/2597 [==============================] - 21s 8ms/step - loss: 0.1307 - acc: 0.9638 - val_loss: 4.7016 - val_acc: 0.3777
Epoch 8/

In [60]:
conv_result4 = conv_model4.evaluate(x_test2, y_test2, batch_size=150)
print(conv_result4)

557/557 [==============================] - 1s 2ms/step
[5.211197773472839, 0.39856373616352114]


In [66]:
conv_result_p_4 = conv_model4.predict(x_test2, batch_size=150)
print(conv_result_p_4)

[[2.1614216e-01 1.8269170e-06 2.6338446e-05 ... 5.0648025e-05
  2.8887248e-04 3.8126094e-05]
 [3.6046094e-10 9.4241299e-09 1.4185495e-09 ... 1.2730460e-06
  2.7120292e-08 9.5570629e-10]
 [2.6239851e-04 3.2038125e-07 1.1406928e-04 ... 3.0498835e-01
  6.1274786e-04 1.0755531e-06]
 ...
 [4.5884686e-04 1.4165133e-07 1.8191237e-04 ... 5.0693416e-06
  4.1005164e-03 4.4900533e-03]
 [8.4080708e-01 2.8532647e-06 3.8869883e-04 ... 4.6513549e-05
  4.5087272e-03 5.6794239e-04]
 [3.4586665e-07 1.9014162e-06 2.3823643e-06 ... 4.9705943e-04
  8.2273714e-02 4.3776050e-05]]


In [68]:
count = 0
for i in range(len(conv_result_p_4)):
    temp = conv_result_p_4[i].argsort()[-3:][::-1]
    if y_test2[i][temp[0]]==1:
        count+=1
    elif y_test2[i][temp[1]]==1:
        count+=1
    elif y_test2[i][temp[2]]==1:
        count+=1
        
print(count/len(y_test2))

0.5637342908438061


In [74]:
x_train3.shape

(2597, 128, 173)